In [1]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  10463      0 --:--:-- --:--:-- --:--:-- 10463
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


In [2]:
 pip install pycuda

     |████████████████████████████████| 1.6MB 5.7MB/s 
     |████████████████████████████████| 61kB 23.5MB/s 
     |████████████████████████████████| 471kB 43.5MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4535222 sha256=09af6a32549e3d303da6c7bb7d462f6bf8b8039713ddbc2e7744c9983f562923
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=0d493708a0a8805720e81721e3e5befee1b4a9f9575ea636520ba58a8e284c0d
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=393f01718450a7a1cf80b2646785de673dd9cfd8ae31658bbe3672587922c065
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [4]:
import numpy

a = numpy.random.randn(4,4)
a = a.astype(numpy.float32)
a_gpu = cuda.mem_alloc(a.nbytes)

cuda.memcpy_htod(a_gpu, a)

mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

[[-4.627452   -2.823246   -2.9386919   0.94816077]
 [-0.78365237 -2.1737494  -1.5743383  -1.0491291 ]
 [ 0.6666753   1.313273   -0.5169166   2.208026  ]
 [ 0.43018326  1.1638689   2.8293772   0.08164818]]
[[-2.313726   -1.411623   -1.4693459   0.47408038]
 [-0.39182618 -1.0868747  -0.78716916 -0.52456456]
 [ 0.33333766  0.6566365  -0.2584583   1.104013  ]
 [ 0.21509163  0.58193445  1.4146886   0.04082409]]
